In [1]:
import pandas as pd
import time
import datetime
import warnings
warnings.filterwarnings("ignore")

I. Read Data

In [2]:
StockNews = pd.read_pickle("2022-10-29_NewsData.pkl")
StockNews = StockNews.reset_index(drop=True)
StockNews['pub_date'] = pd.to_datetime(StockNews['pub_date'], format="%Y-%m-%d")

In [3]:
StockNews.describe()

,search_time,search_key,title,link,pub_date,description,source,content
count,2798,2798,2798,2798,2798,2798,2798,2798
unique,201,149,1421,1452,1299,1458,10,1412
top,2022-11-04 05:33:03,智原,〈台股盤中〉台積、傳產壓盤 一度跌近200點失守萬三及月線 - Yahoo奇摩股市,https://news.google.com/__i/rss/rd/articles/CB...,2022-11-03 04:49:24,"<a href=""https://news.google.com/__i/rss/rd/ar...",Yahoo奇摩股市,美國聯準會 (FED) 升息 3 碼，美元續強，資金持續匯出，台股在台積電及傳產走弱下，開盤...
freq,39,78,28,28,30,28,789,30
first,NaN,NaN,NaN,NaN,2022-10-29 00:10:02,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2022-11-03 21:58:58,NaN,NaN,NaN


In [6]:
# remove chinese stop words
from TCSP import read_stopwords_list
stop_words=read_stopwords_list()

In [7]:
text = StockNews['content']
corpus = ', '.join(text)
corpus = ''.join([i for i in corpus if not i.isdigit()])
corpus = ''.join([i for i in corpus if i not in stop_words])
corpus = corpus.replace(" ", "")
corpus[:500]

'鴻海-TW晚間宣布攜手沙烏拉伯公投資基金PublcInveenFunPIF合資首沙國國家電動車品牌CEER沙國生產電動車包括轎車休旅車預計開賣鎖定沙國東北市場Ceer提供系列電動車設計製造銷售服務運BMW授權部件技術進行車輛開發鴻海開發車輛電子電氣架構智座艙智網關動輔助駕駛技術領域提供工程產品服務輛車沙國設計製造全球高規格汽車品質安全標準進行測試CeerPIF透投資前景產業實現沙國GDP增長元化戰略部預計吸引超.億美元外國直接投資創造達萬直接間接業機預計沙國GDP貢獻億美元沙烏拉伯總理兼公投資基金PIF主席穆罕默德MhbnSlnbnAbulzz表示沙烏拉伯僅造新汽車品牌正燃新產業生態系統吸引國際投資創造民間業機鴻海董事長劉揚偉表示利鴻海科技專長支持Ceer願景造系列具備智網關通訊娛樂動輔助駕駛功備受歡迎電動車透Ceer鴻海目標沙烏拉伯周電動車造成市場主流產品財訊快報／記劉居全報導針市場傳出國令鴻海鄭州工廠周圍域進行期天封鎖引發市場Phne生產擔憂歐系外資新出爐報告表示解期天封城期間鴻海鄭州園保持閉環生產出貨歐系外資認導短期供應斷鴻海季銷售估機構預測影響應限歐系外資維持鴻海季營收季增增表'

In [8]:
# Word segmentation
import jieba
import jieba.analyse

In [9]:
# TFIDF importance
tags = jieba.analyse.extract_tags(corpus,topK=1000)
with open(r'TFIDF_1000.txt', 'w') as fp:
    for item in tags:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 1.006 seconds.
Prefix dict has been built successfully.


Done


In [10]:
# frequency importance
seg_list = list(jieba.cut(corpus,use_paddle=True))

In [11]:
print(len(seg_list))
seg_list[:10]

927358


['鴻海', '-', 'TW', '晚間', '宣布', '攜手', '沙烏', '拉伯', '公投', '資']

In [12]:
word_dict = {}
# with open("wordCount.txt", 'w') as wf2:  # 打开文件  

for item in seg_list:
    if item not in word_dict:  # 统计数量  
        word_dict[item] = 1
    else:
        word_dict[item] += 1


In [35]:
orderList={k: v for k, v in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)}
del orderList['.']
del orderList['-']
del orderList['\t']
del orderList['/']
del orderList['.%']
del orderList[':']
del orderList['..']
del orderList['~']
del orderList['IC']
del orderList['Fe']
del orderList['Q']
del orderList['受']
del orderList['係']
del orderList['金']
del orderList['陳']
del orderList['出']
del orderList['程']
del orderList['季']
del orderList['應']
del orderList['／']
del orderList['%']
del orderList['元']
del orderList['TW']
del orderList['KY']
del orderList['EPS']
del orderList['台北']
del orderList['半導體']
del orderList['伺服器']
del orderList['億']
del orderList['黃']
del orderList['台幣']
del orderList['政府']
del orderList['*']
del orderList['億元']
del orderList['市場']
del orderList['公司']
del orderList['表示']

In [36]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

orderList = take(1000, orderList.items())
orderList[:10]

[('長', 2601),
 ('預期', 2480),
 ('跌', 2437),
 ('新', 2421),
 ('指數', 2243),
 ('成長', 2239),
 ('買', 2215),
 ('需求', 2094),
 ('漲', 2065),
 ('升息', 2059)]

In [50]:
orderList_1000

['長',
 '預期',
 '跌',
 '新',
 '指數',
 '成長',
 '買',
 '需求',
 '漲',
 '升息',
 '達',
 '股價',
 '逾',
 '投資',
 '全球',
 '新高',
 '持續',
 '台',
 '目前',
 '美元',
 '台股',
 '營收',
 '台積',
 '股',
 '指出',
 '製',
 '反彈',
 '影響',
 '利率',
 '金融',
 '獲利',
 '族群',
 '併',
 '美國',
 '台灣',
 '公布',
 '廠',
 '約',
 '高',
 '前季',
 '股市',
 '增加',
 '復',
 '低',
 '電子',
 '客戶',
 '科技',
 '晶片',
 '產業',
 '跌幅',
 '純益',
 '疫情',
 '行情',
 '包括',
 '董事',
 '相關',
 '前',
 '近期',
 '毛利率',
 '表現',
 '賺',
 '超',
 '產品',
 '企業',
 '基金',
 '線',
 '未',
 '交易',
 '受惠',
 '站',
 '美股',
 '財報',
 '收',
 '股票',
 '聯準',
 '走高',
 '近',
 '賣',
 '提供',
 '業',
 '幅',
 '外資',
 '整體',
 '檔',
 '開始',
 '預估',
 '布局',
 '政策',
 '合作',
 '集團',
 '宣布',
 '造',
 '閱讀',
 '漲幅',
 '發展',
 '報導',
 '庫存',
 '銀行',
 '提升',
 '顯示',
 '帶動',
 '預計',
 '方面',
 '推出',
 '出現',
 '走',
 '沒',
 '支出',
 '目標',
 '母公司',
 '臺',
 '營業',
 '單季',
 '平台',
 '期',
 '成交量',
 '價格',
 '強勢',
 '面板',
 '終場',
 '利空',
 '季營',
 '消費',
 '壓力',
 '持股',
 '增',
 '間',
 '進入',
 '操作',
 'ETF',
 '基面',
 '報告',
 '代工',
 '減少',
 '景氣',
 '衰退',
 '完成',
 'ABF',
 '調整',
 '次',
 '保守',
 '問題',
 '歷史',
 '公告',
 '盤',
 '短期',
 '平均',
 '投

In [47]:
sz = len(orderList)
orderList_1000=[]
for i in range(sz):
    word = orderList[i][0].strip('\n')
    orderList_1000.append(word)

In [49]:
# Word Frequency
with open(r'Top1000FrequencyWords.txt', 'w', encoding='utf-8') as fp:
    for item in orderList_1000:
        # write each item on a new line
        fp.write("%s," % item)
    print('Done')    

Done
